In [1]:
import pandas as pd
import matplotlib as plt
import seaborn as sns

In [2]:
df1 = pd.read_csv("Telco-Customer-Churn.csv")

In [3]:
df1.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [5]:
df1.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [6]:
!pip install --upgrade langchain_experimental

In [7]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType


In [14]:
import os
print(os.getenv("OPENAI_API_KEY"))

None


In [11]:
# Initialize the ChatOpenAI model
model = ChatOpenAI(temperature=0, model="gpt-4")

agent = create_pandas_dataframe_agent(
    model,
    df1,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True
)

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [50]:
from io import BytesIO
import base64

In [51]:
def generate_visualization(query):
    # This function should parse the user's query and generate the corresponding visualization.
    # Here we'll just create a dummy plot for illustration.
    plt.figure(figsize=(10, 6))
    sns.histplot(df1['some_column'])
    plt.title('Sample Histogram')
    plt.xlabel('Values')
    plt.ylabel('Frequency')
    
    # Save the plot to a BytesIO object and encode it as a base64 string
    buf = BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img_base64 = base64.b64encode(buf.getvalue()).decode('utf-8')
    buf.close()
    
    return img_base64

In [59]:
def handle_user_query(query):
    # Process the query using the agent
    response = agent(query)
    
    # Generate a visualization based on the response or query
    visualization = generate_visualization(query)
    
    # Create a report including the visualization
    report = f"""
    <html>
    <head><title>Data Analysis Report</title></head>
    <body>
    <h1>Analysis Report</h1>
    <p>Query: {query}</p>
    <p>Response: {response}</p>
    <h2>Visualization</h2>
    <img src="data:image/png;base64,{visualization}" alt="Visualization">
    </body>
    </html>
    """
    
    return report

In [62]:
!pip install streamlit


  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/7c/2e/60e624bbe16f4baa45cb6e48a1ee05edd48a0a14cceec4d7eec9258755ac/streamlit-1.38.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for altair<6,>=4.0 from https://files.pythonhosted.org/packages/9b/52/4a86a4fa1cc2aae79137cc9510b7080c3e5aede2310d14fae5486feec7f7/altair-5.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for blinker<2,>=1.0.0 from https://files.pythonhosted.org/packages/bb/2a/10164ed1f31196a2f7f3799368a821765c62851ead0e630ab52b8e14b4d0/blinker-1.8.2-py3-none-any.whl.metadata
  Obtaining dependency information for cachetools<6,>=4.0 from https://files.pythonhosted.org/packages/a4/07/14f8ad37f2d12a5ce41206c21820d8cb6561b728e51fad4530dff0552a67/cachetools-5.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for gitpython!=3.1.19,<4,>=3.0.7 from https://files.pythonhosted.org/packages/e9/bd/cc3a402a6439c15c3d4294333e13042b915bbeab54edc4

In [63]:
import streamlit as st
import pandas as pd

st.title('Data Analysis with LangChain')

# Upload CSV file
uploaded_file = st.file_uploader("Upload CSV", type="csv")
if uploaded_file:
    df1 = pd.read_csv(uploaded_file)
    st.write("Data Preview:", df1.head())
    
    # Get user query
    query = st.text_input("Enter your analysis query:")
    
    if st.button("Generate Report"):
        if query:
            report = handle_user_query(query)
            
            # Display the report
            st.markdown(report, unsafe_allow_html=True)
            
            # Provide an option to download the report
            with open('report.html', 'w') as file:
                file.write(report)
            st.download_button(
                label="Download Report",
                data=open('report.html', 'r').read(),
                file_name='report.html',
                mime='text/html'
            )
        else:
            st.warning("Please enter a query.")

2024-09-09 16:52:34.557 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-09 16:52:34.746 
  command:

    streamlit run C:\Users\irumj\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-09-09 16:52:34.747 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-09 16:52:34.747 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-09 16:52:34.748 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-09 16:52:34.751 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-09 16:52:34.751 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


SyntaxError: invalid syntax (421834696.py, line 1)